# Flag Pattern Detection

In [1]:
import pandas as pd
data = pd.read_parquet("/liubinxu/liubinxu/finance/learning/data/510050.qfq.kdj.parquet")
data


,open,close,high,low,vol,amount,datetime,code,date,date_stamp,...,close_rolling_96_mean_amount_normalize60_rolling_96_mean_corr_roll480,close_rolling_480_mean_amount_normalize60_rolling_480_mean_corr_roll24,close_rolling_480_mean_amount_normalize60_rolling_480_mean_corr_roll96,close_rolling_480_mean_amount_normalize60_rolling_480_mean_corr_roll480,K,D,J,target_close1,target_close2,target_close5
index,,,,,,,,,,,,,,,,,,,,,
2021-03-11 09:40:00,3.461653,3.456786,3.471388,3.451918,22699100.0,80798040.0,2021-03-11 09:40:00,510050,2021-03-11,1.615392e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.017460,0.006759,0.011828
2021-03-11 09:45:00,3.458733,3.472361,3.476255,3.458733,20905000.0,74504384.0,2021-03-11 09:45:00,510050,2021-03-11,1.615392e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.014298,0.005607,0.005607
2021-03-11 09:50:00,3.472361,3.494751,3.495724,3.471388,34468600.0,123415776.0,2021-03-11 09:50:00,510050,2021-03-11,1.615392e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.008914,-0.003343,0.001671
2021-03-11 09:55:00,3.496698,3.498645,3.503512,3.495724,31407400.0,112923328.0,2021-03-11 09:55:00,510050,2021-03-11,1.615392e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005843,-0.002226,0.000278
2021-03-11 10:00:00,3.497671,3.500592,3.510326,3.496698,22805200.0,82089312.0,2021-03-11 10:00:00,510050,2021-03-11,1.615392e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.005006,-0.001112,-0.000556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-12 14:40:00,2.637000,2.634000,2.637000,2.632000,13134900.0,34614032.0,2023-05-12 14:40:00,510050,2023-05-12,1.683821e+09,...,0.010836,0.838282,-0.852782,-0.475668,15.220687,16.948490,11.765081,NaN,NaN,NaN
2023-05-12 14:45:00,2.634000,2.633000,2.634000,2.632000,18961400.0,49925864.0,2023-05-12 14:45:00,510050,2023-05-12,1.683821e+09,...,0.011245,0.891564,-0.841253,-0.467698,13.874382,16.436139,8.750869,NaN,NaN,NaN
2023-05-12 14:50:00,2.633000,2.634000,2.635000,2.633000,19275000.0,50766704.0,2023-05-12 14:50:00,510050,2023-05-12,1.683821e+09,...,0.011613,0.917682,-0.828578,-0.459640,13.942937,16.020605,9.787602,NaN,NaN,NaN


In [2]:
df = data[['open', 'high', 'low', 'close', 'volume']]

In [3]:

# df = pd.read_csv("EURUSD_Candlestick_1_Hour_BID_01.07.2020-15.07.2023.csv")
# df.columns=['time', 'open', 'high', 'low', 'close', 'volume']
#Check if NA values are in data
# df=df[df['volume']!=0]

df['time'] = df.index
df.index
df.isna().sum()
#设置数字的索引
df.reset_index(drop=True, inplace=True)

/tmp/ipykernel_3900674/3655515948.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time'] = df.index


In [150]:
df.iloc[77, ]

open                       3.5113
high                     3.512273
low                      3.502539
close                    3.504486
volume                 19809500.0
time          2021-03-12 13:35:00
pivot                           1
pointpos                 3.501539
flag                            0
pivot24                         0
pointpos24                    NaN
Name: 77, dtype: object

In [60]:
def pivotid(df1, l, n1, n2): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    

    for i in range(l-n1, l+n2+1):
#         if l == 77:
#             print(df1.low[i], df.low[l]) 
        if(df1.low[l]>df1.low[i]):
            pividlow=0
        if(df1.high[l]<df1.high[i]):
            pividhigh=0
    
#     # 添加一下其它极值点
#     if pividlow == 0:
#         low_min = min(df1.low[l-12: l+3])
#         low_min_idx = df1.low[l-12: l+3].idxmin()
#         if df1.low[l] == min(df1.low[l-3: l+3]): 
#             if df1.low[l] - low_min <  low_min * 0.002 and abs(l - low_min_idx) > 5:
# #                 print("##### ", l)
#                 pividlow = 1
#     if pividhigh == 0:
#         high_max = max(df1.low[l-12: l+3])
#         high_max = df1.low[l-12: l+3].idxmax()
#         if df1.high[l] == max(df1.high[l-3: l+3]): 
#             if high_max - df1.high[l] < high_max * 0.002 and abs(l - high_max_idx) > 5:
#                 pividhigh = 1
    if pividlow and pividhigh:
        return 3
    elif pividlow:
        return 1
    elif pividhigh:
        return 2
    else:
        return 0
    
df['pivot'] = df.apply(lambda x: pivotid(df, x.name,6,6), axis=1)
df['pivot24'] = df.apply(lambda x: pivotid(df, x.name,24,24), axis=1)
df['pivot120'] = df.apply(lambda x: pivotid(df, x.name,120,72), axis=1)

/tmp/ipykernel_3900674/76947359.py:40: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3900674/76947359.py:41: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3900674/76947359.py:42: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [5]:
def pivot_merge(df1, l, n1, n2, pct): #n1 n2 before and after candle l
    if l-n1 < 0 or l+n2 >= len(df1):
        return 0
    
    pividlow=1
    pividhigh=1
    
    if df['pivot24'][l] != 0:
        return df['pivot24'][l]
    
    elif df['pivot'][l] == 1:
        if (df1.low[l-n1] + df1.low[l+n2])/2 - df1.low[l] > (df1.low[l] *pct):
#             print(df[l-n1: l+ n2])
            return 1
            
        else:
            return 0
        
    elif df['pivot'][l] == 2:
        if (df1.high[l-n1] + df1.high[l+n2])/2 - df1.high[l] < - (df1.low[l] *pct):
#             print(df[l-n1: l+ n2])
            return 2
            
        else:
            return 0
    
    else:
        return 0 

df['pivot_merge'] = df.apply(lambda x: pivot_merge(df, x.name,3,3, 0.005), axis=1)

/tmp/ipykernel_3900674/1632474626.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pivot_merge'] = df.apply(lambda x: pivot_merge(df, x.name,3,3, 0.005), axis=1)


In [453]:
df['pivot_merge']

0        0
1        0
2        0
3        0
4        0
        ..
25290    0
25291    0
25292    0
25293    0
25294    0
Name: pivot_merge, Length: 25295, dtype: int64

In [6]:
def pivot_clean(df1, l, n): #n1 n2 before and after candle l
    if l-n < 0 or l+n >= len(df1):
        pass
    
    
    if df['pivot_merge'][l] == 1:
        if len(df[df['pivot_merge']==1].loc[l-n:l+n]) != 1:
            min_idx = df[df['pivot_merge']==1].loc[l-n:l+n]["low"].idxmin
            idxs = [idx for idx in df[df['pivot_merge']==1].loc[l-n:l+n].index if idx != min_idx]
            for idx in idxs:
                df['pivot_merge'][idx] = 0
        else:
            pass
        
    elif df['pivot_merge'][l] == 2:
        if len(df[df['pivot_merge']==2].loc[l-n:l+n]) != 1:
            max_idx = df[df['pivot_merge']==2].loc[l-n:l+n]["high"].idxmax
            idxs = [idx for idx in df[df['pivot_merge']==2].loc[l-n:l+n].index if idx != max_idx]
            # print(l, idxs)
            for idx in idxs:
                df['pivot_merge'][idx] = 0
        else:
            pass
    
    else:
        pass

df.apply(lambda x: pivot_clean(df, x.name, 8), axis=1)

0        None
1        None
2        None
3        None
4        None
         ... 
25290    None
25291    None
25292    None
25293    None
25294    None
Length: 25295, dtype: object

In [430]:
idxs = [idx for idx in df[df['pivot_merge']==1].loc[157:800].index if idx != 192]
idxs
df.loc[idxs]

,open,high,low,close,volume,time,pivot,pointpos,flag,pivot24,pointpos24,pivot_merge,pointpos_merge
157,3.449971,3.460680,3.448998,3.457759,4041100.0,2021-03-16 10:45:00,1,3.447998,0,0,NaN,1,3.447998
327,3.394484,3.396431,3.389616,3.394484,10292500.0,2021-03-19 14:25:00,1,3.388616,0,1,3.388616,1,3.388616
407,3.381829,3.381829,3.375014,3.377935,16552200.0,2021-03-23 13:05:00,1,3.374014,1,1,3.374014,1,3.374014
481,3.338023,3.359439,3.338023,3.356518,12925200.0,2021-03-25 09:45:00,1,3.337023,0,1,3.337023,1,3.337023
521,3.351651,3.352625,3.348731,3.352625,2117200.0,2021-03-25 14:35:00,1,3.347731,0,1,3.347731,1,3.347731
577,3.408112,3.418820,3.407139,3.417847,10551100.0,2021-03-29 09:45:00,1,3.406139,0,1,3.406139,1,3.406139
614,3.423688,3.426608,3.421741,3.426608,3732300.0,2021-03-29 14:20:00,1,3.420741,0,1,3.420741,1,3.420741
689,3.413953,3.419794,3.410059,3.418820,3794400.0,2021-03-31 11:05:00,1,3.409059,0,1,3.409059,1,3.409059
749,3.445104,3.448024,3.441210,3.442184,3073500.0,2021-04-01 13:35:00,1,3.440210,0,1,3.440210,1,3.440210
750,3.443157,3.446078,3.441210,3.446078,1246900.0,2021-04-01 13:40:00,1,3.440210,0,1,3.440210,1,3.440210


In [462]:
len(df[df['pivot_merge']==2])

380

In [242]:
df[df["pivot"] != df["pivot24"]][df["pivot_merge"] != 0]

/tmp/ipykernel_1333883/1338934189.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,open,high,low,close,volume,time,pivot,pointpos,flag,pivot24,pointpos24,pivot_merge,pointpos_merge
97,3.480149,3.492804,3.476255,3.491831,25341300.0,2021-03-15 09:45:00,1,3.475255,0,0,3.475255,1,3.475255
157,3.449971,3.460680,3.448998,3.457759,4041100.0,2021-03-16 10:45:00,1,3.447998,0,0,NaN,1,3.447998
286,3.494751,3.497671,3.493777,3.495724,8113500.0,2021-03-18 15:00:00,2,3.498671,0,0,NaN,2,3.498671
446,3.382802,3.395457,3.380855,3.385722,10769000.0,2021-03-24 10:50:00,2,3.396457,0,0,NaN,2,3.396457
528,3.390590,3.398378,3.388643,3.396431,15529300.0,2021-03-26 09:40:00,2,3.399378,0,0,3.399378,2,3.399378
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22078,2.823000,2.825000,2.822000,2.824000,12156200.0,2023-02-02 15:00:00,2,2.826000,0,0,2.826000,2,2.826000
22128,2.737000,2.737000,2.731000,2.733000,33417300.0,2023-02-06 09:40:00,1,2.730000,0,0,2.730000,1,2.730000
22415,2.776000,2.784000,2.768000,2.774000,21931700.0,2023-02-14 09:35:00,3,NaN,0,2,NaN,2,NaN
24671,2.708000,2.709000,2.692000,2.693000,46822000.0,2023-04-21 09:35:00,3,NaN,0,2,NaN,2,NaN


In [10]:
df[24660:24680]

,open,high,low,close,volume,time,pivot
24660,2.703,2.705,2.703,2.704,9614500.0,2023-04-20 14:10:00,0
24661,2.705,2.708,2.704,2.707,18703600.0,2023-04-20 14:15:00,2
24662,2.708,2.708,2.705,2.707,11796900.0,2023-04-20 14:20:00,2
24663,2.707,2.707,2.704,2.705,9105800.0,2023-04-20 14:25:00,0
24664,2.706,2.707,2.705,2.707,13151000.0,2023-04-20 14:30:00,0
24665,2.707,2.708,2.705,2.705,13469200.0,2023-04-20 14:35:00,2
24666,2.706,2.707,2.704,2.704,10498200.0,2023-04-20 14:40:00,0
24667,2.704,2.704,2.702,2.704,12896200.0,2023-04-20 14:45:00,1
24668,2.704,2.704,2.702,2.703,12598300.0,2023-04-20 14:50:00,0
24669,2.704,2.706,2.703,2.706,20700000.0,2023-04-20 14:55:00,0


## Pivot Candles Vizualisation

In [61]:
import numpy as np
def pointpos(x, window=None):
    
    if x[window]==1:
        return x['low']-1e-3
    elif x[window]==2:
        return x['high']+1e-3
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row, 'pivot'), axis=1)
df['pointpos24'] = df.apply(lambda row: pointpos(row, 'pivot24'), axis=1)
df['pointpos120'] = df.apply(lambda row: pointpos(row, 'pivot120'), axis=1)
df['pointpos_merge'] = df.apply(lambda row: pointpos(row, 'pivot_merge'), axis=1)

/tmp/ipykernel_3900674/354700584.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3900674/354700584.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_3900674/354700584.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipyke

In [8]:
df[np.isnan(df["pointpos_merge"]) == False]

,open,high,low,close,volume,time,pivot,pivot24,pivot_merge,pointpos,pointpos24,pointpos_merge
39,3.489884,3.493777,3.481122,3.485016,12811600.0,2021-03-11 14:25:00,1,1,1,3.480122,3.480122,3.480122
47,3.534663,3.534663,3.522982,3.522982,24269600.0,2021-03-12 09:35:00,2,2,2,3.535663,3.535663,3.535663
84,3.522982,3.526875,3.521035,3.522982,4917500.0,2021-03-12 14:10:00,2,2,2,3.527875,3.527875,3.527875
97,3.480149,3.492804,3.476255,3.491831,25341300.0,2021-03-15 09:45:00,1,0,1,3.475255,NaN,3.475255
136,3.434396,3.447051,3.432449,3.443157,4554400.0,2021-03-15 14:30:00,1,1,1,3.431449,3.431449,3.431449
...,...,...,...,...,...,...,...,...,...,...,...,...
24973,2.694000,2.698000,2.694000,2.696000,22774900.0,2023-05-04 10:45:00,2,2,2,2.699000,2.699000,2.699000
25019,2.683000,2.684000,2.675000,2.677000,13972300.0,2023-05-05 10:35:00,1,1,1,2.674000,2.674000,2.674000
25075,2.723000,2.730000,2.722000,2.727000,32056100.0,2023-05-08 11:15:00,2,2,2,2.731000,2.731000,2.731000
25183,2.678000,2.678000,2.673000,2.678000,12883000.0,2023-05-10 13:45:00,1,1,1,2.672000,2.672000,2.672000


,open,high,low,close,volume,time,pivot,pivot24,pivot_merge,pointpos,pointpos24,pointpos_merge
12800,2.883097,2.885069,2.878167,2.879153,25573900.0,2022-04-15 13:50:00,0,0,0,NaN,NaN,NaN
12801,2.879153,2.882111,2.878167,2.878167,24243300.0,2022-04-15 13:55:00,0,0,0,NaN,NaN,NaN
12802,2.878167,2.881125,2.876195,2.880139,22992500.0,2022-04-15 14:00:00,0,0,0,NaN,NaN,NaN
12803,2.880139,2.885069,2.879153,2.881125,22725000.0,2022-04-15 14:05:00,0,0,0,NaN,NaN,NaN
12804,2.880139,2.881125,2.878167,2.881125,19252800.0,2022-04-15 14:10:00,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
13995,2.683923,2.683923,2.678993,2.679979,2885500.0,2022-05-25 13:25:00,0,0,0,NaN,NaN,NaN
13996,2.679979,2.679979,2.675049,2.675049,11928700.0,2022-05-25 13:30:00,0,0,0,NaN,NaN,NaN
13997,2.675049,2.677021,2.675049,2.677021,16770300.0,2022-05-25 13:35:00,0,0,0,NaN,NaN,NaN
13998,2.676035,2.677021,2.673077,2.674063,17684000.0,2022-05-25 13:40:00,1,0,0,2.672077,NaN,NaN


In [69]:
ranges_list

[{'low_start': 192,
  'high_start': 383,
  'idxlows': [192, 327, 407, 481],
  'idxhighs': [383, 432, 446, 489],
  'lows': [3.440236708619679,
   3.3896163608980436,
   3.375014337516799,
   3.3380225449509897],
  'highs': [3.440236708619665,
   3.424661217013005,
   3.3954571702505274,
   3.375987805742208],
  'slmin': -0.00033910674921420396,
  'intercmin': 3.505003287032474,
  'rmin': -0.9906781059325097,
  'slmax': -0.0006294811127493567,
  'intercmax': 3.6844837122341945,
  'rmax': -0.9536867008386456},
 {'low_start': 481,
  'high_start': 489,
  'idxlows': [481, 521, 577, 614],
  'idxhighs': [489, 528, 587, 644],
  'lows': [3.3380225449509897,
   3.3487306954305676,
   3.4071387889555176,
   3.4217408123367496],
  'highs': [3.375987805742208,
   3.3983775749267773,
   3.4519183273246457,
   3.4713876918329536],
  'slmin': 0.0006913052156965261,
  'intercmin': 2.9999001259128355,
  'rmin': 0.977039641524949,
  'slmax': 0.0006476652845689411,
  'intercmax': 3.060429960028901,
  'rmax

In [95]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
from_d = 20000
to_d = 24000
dfpl = df[from_d:to_d]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])



fig.add_scatter(x=dfpl.index, y=dfpl['pointpos_merge'], mode="markers",
                 marker=dict(size=5, color="MediumPurple"),
                 name="pivot_merge")
# fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
#                  marker=dict(size=5, color="black"),
#                  name="pivot")

for range_dic in ranges_list:
    idxlows = np.array(range_dic["idxlows"])
    slmin = range_dic["slmin"]
    intercmin = range_dic["intercmin"]
    idxhighs = np.array(range_dic["idxhighs"])
    slmax = range_dic["slmax"]
    intercmax = range_dic["intercmax"]
#     print(idxlows)
    if min(idxlows) > from_d and max(idxlows) < to_d:
        fig.add_trace(go.Scatter(x=idxlows, y=slmin*idxlows + intercmin, mode='lines', name='min slope', fillcolor="MediumPurple"))
        fig.add_trace(go.Scatter(x=idxhighs, y=slmax*idxhighs + intercmax, mode='lines', name='max slope', fillcolor="MediumPurple")) 

#fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [473]:
?go.Scatter

In [363]:
ranges_list[20]

{'low_start': 5297,
 'high_start': 5150,
 'idxlows': [5297, 5328, 5361],
 'idxhighs': [5150, 5186, 5257, 5385, 5387, 5474],
 'lows': [3.0800534652154603, 3.04792901377674, 2.9992556025059525],
 'highs': [3.197843120490768,
  3.1929757793636937,
  3.157930923248723,
  5385,
  5387,
  5474],
 'slmin': -0.0012647484184745883,
 'intercmin': 9.78183543307764,
 'rmin': -0.9950374934440367,
 'slmax': 0.0006855870192162652,
 'intercmax': -0.6271124529029981,
 'rmax': 0.9998063579133049}

In [354]:
linregress([2210, 2275, 2311, 2448, 2496], [3.3925367655741874,
  3.3720939328404556,
  3.3565184412337996,
  3.522981507779887,
  3.5366100629357065])

LinregressResult(slope=0.0006451848700730593, intercept=1.9212540671412641, rvalue=0.8946732162054151, pvalue=0.04037931423553993, stderr=0.0001859926063682588, intercept_stderr=0.4371670337145161)

In [341]:
1.555 * np.array([1,2,3])

array([1.555, 3.11 , 4.665])

In [152]:
dfchoose = df[:120]
dfchoose['pivot24'] = dfchoose.apply(lambda x: pivotid(dfchoose, x.name,24,6), axis=1)

3.5122733573004608 3.502538675046302
3.5064325479479654 3.502538675046302
3.504485611497134 3.502538675046302
3.5093529526242135 3.502538675046302
3.5054590797225504 3.502538675046302
3.4996182703700556 3.502538675046302
3.4918305245667285 3.502538675046302
3.4918305245667285 3.502538675046302
3.4937774610175603 3.502538675046302
3.487936651665065 3.502538675046302
3.487936651665065 3.502538675046302
3.4908570563413117 3.502538675046302
3.497671333919223 3.502538675046302
3.496697865693807 3.502538675046302
3.5035121432717187 3.502538675046302
3.500591738595471 3.502538675046302
3.499618270370055 3.502538675046302
3.511299889075046 3.502538675046302
3.5112998890750453 3.502538675046302
3.5112998890750444 3.502538675046302
3.5064325479479646 3.502538675046302
3.506432547947965 3.502538675046302
3.508379484398797 3.502538675046302
3.5074060161733813 3.502538675046302
3.502538675046302 3.502538675046302
3.504485611497134 3.502538675046302
3.5083794843987968 3.502538675046302
3.50935295262

/tmp/ipykernel_1333883/4258345769.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [89]:
def filter_inner_point(x, y, type="high"):

    # 创建示例数据，包括一些离群点
    x = np.array(x)
    y = np.array(y)

    # 创建 RANSACRegressor 模型
    slmax_add, intercmax_add, rmax_add, _, _ = linregress(x, y)
    y_line = slmax_add * x + intercmax_add
    if type=="low":
        max_c = max((y - y_line)/y)
        if max_c > 0.001:
            choose = (y - y_line) /y != max_c
            x_choose = list(x[choose])
            y_choose = list(y[choose])
        else:
            x_choose = list(x)
            y_choose = list(y)
    else:
        max_c = max((y_line -y) /y)
        if max_c > 0.001:
            choose = (y_line - y) /y != max_c
            x_choose = list(x[choose])
            y_choose = list(y[choose])
        else:
            x_choose = list(x)
            y_choose = list(y)



    linregress(x_choose, y_choose)
    return slmax_add, intercmax_add, rmax_add, x_choose, y_choose

In [153]:
dfchoose.iloc[77,]

open                       3.5113
high                     3.512273
low                      3.502539
close                    3.504486
volume                 19809500.0
time          2021-03-12 13:35:00
pivot                           1
pointpos                 3.501539
flag                            0
pivot24                         0
pointpos24                    NaN
Name: 77, dtype: object

In [90]:
from scipy.stats import linregress
ranges_list  = list()

def detect_flag(candle, backcandles, window, pivot_window, plot_flag=False):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    localdf = df[candle-backcandles-window:candle-window]  
    highs = list(localdf[localdf[pivot_window] == 2].high.tail(3).values)
    idxhighs = list(localdf[localdf[pivot_window] == 2].high.tail(3).index)
    lows = list(localdf[localdf[pivot_window] == 1].low.tail(3).values)
    idxlows = list(localdf[localdf[pivot_window] == 1].low.tail(3).index)
#     if candle == 1122:
#         print(idxlows, lows)
#         print(idxhighs, highs)
    if len(highs) == 3 and len(lows) == 3:
#         order_condition = ( 
#             (idxlows[0] < idxhighs[0] 
#             < idxlows[1] < idxhighs[1] 
#             < idxlows[2] < idxhighs[2]) 
#             or 
#             (idxhighs[0] < idxlows[0] 
#              < idxhighs[1] < idxlows[1] 
#              < idxhighs[2] < idxlows[2]) )
        order_condition = (idxlows[2] > idxhighs[0] or idxhighs[2]> idxlows[0] )   
        
        slmin, intercmin, rmin, _, _ = linregress(idxlows, lows)
        slmax, intercmax, rmax, _, _ = linregress(idxhighs, highs)

        if (order_condition and 
            (rmax*rmax)>=0.5 
            and (rmin*rmin)>=0.5 
#             and slmin>=0.0001 
#             and slmax<=-0.0001
        ):
            #and ((abs(slmin)-abs(slmax))/abs(slmax)) < 0.05):

            if plot_flag:
                fig = go.Figure(data=[go.Candlestick(x=localdf.index,
                open=localdf['open'],
                high=localdf['high'],
                low=localdf['low'],
                close=localdf['close'])])

                fig.add_scatter(x=localdf.index, y=localdf['pointpos'], mode="markers",
                marker=dict(size=10, color="MediumPurple"),
                name="pivot")
                fig.add_trace(go.Scatter(x=idxlows, y=slmin*idxlows + intercmin, mode='lines', name='min slope'))
                fig.add_trace(go.Scatter(x=idxhighs, y=slmax*idxhighs + intercmax, mode='lines', name='max slope'))
                fig.update_layout(
                xaxis_rangeslider_visible=False,
                plot_bgcolor='white', # change the background to white
                xaxis=dict(showgrid=True, gridcolor='white'), # change the x-axis grid to white
                yaxis=dict(showgrid=True, gridcolor='white') # change the y-axis grid to white
                )
                fig.show()
            if len(ranges_list) > 0:
                ranges_last = ranges_list[-1]
                
                if ranges_last["idxlows"][-1] == idxlows[-1] and ranges_last["idxhighs"][-1] == idxhighs[-1]:
                    pass
                else:
                    # print(idxlows[0])
                    # print(idxhighs[0])
                    if ranges_last["idxlows"][-1] == idxlows[-2]:
                        print("add_idx {}".format(idxlows[0]))
                        idxlows_add = ranges_last["idxlows"] + [idxlows[-1]]
                        lows_add = ranges_last["lows"] + [lows[-1]]
                        slmin_add, intercmin_add, rmin_add, _, _ = linregress(idxlows_add, lows_add)
                        slmin_add_c, intercmin_add_c, rmin_add_c, xchoose, ychoose = filter_inner_point(idxlows_add, lows_add, type="low")
                        
                        if (rmin_add*rmin_add)>=0.95:
                            ranges_last.update({
                                "idxlows": idxlows_add,
                                "lows":lows_add,
                                "slmin": slmin_add,
                                "intercmin":intercmin_add,
                                "rmin": rmin_add,
                            })
                        elif rmin_add_c > ranges_last["rmin"]:
                            ranges_last.update({
                                "idxlows": xchoose,
                                "lows":ychoose,
                                "slmin": slmin_add_c,
                                "intercmin":intercmin_add_c,
                                "rmin": rmin_add_c,
                            })                        
                        else:
                            ranges_list.append(
                                {
                                    "low_start": idxlows[0],
                                    "high_start":idxhighs[0],
                                    "idxlows": list(idxlows),
                                    "idxhighs": list(idxhighs),
                                    "lows":lows,
                                    "highs": highs,
                                    "slmin": slmin,
                                    "intercmin":intercmin,
                                    "rmin": rmin,
                                    "slmax": slmax,
                                    "intercmax": intercmax,
                                    "rmax":rmax
                                }
                            )
                        
                    elif ranges_last["idxhighs"][-1] == idxhighs[-2]:                        
                        idxhighs_add = ranges_last["idxhighs"] + [idxhighs[-1]]
                        highs_add = ranges_last["highs"] + [highs[-1]]
                        slmax_add, intercmax_add, rmax_add, _, _ = linregress(idxhighs_add, highs_add)
                        slmax_add_c, intercmax_add_c, rmax_add_c, xmchoose, ymchoose = filter_inner_point(idxhighs_add, highs_add, type="high")
                        print(xmchoose, rmax_add_c, ranges_last["rmax"])
                        
                        if (rmax_add*rmax_add)>=0.95:
                            ranges_last.update({
                                "idxhighs": idxhighs_add,
                                "highs":highs_add,
                                "slmax": slmax_add,
                                "intercmax":intercmax_add,
                                "rmax": rmax_add,
                        })
                        elif rmax_add_c > ranges_last["rmax"]:                            
                            ranges_last.update({
                                "idxhighs": xmchoose,
                                "highs":ymchoose,
                                "slmax": slmax_add_c,
                                "intercmax":intercmax_add_c,
                                "rmax": rmax_add_c,
                            }) 
                        else:
                            ranges_list.append(
                                {
                                    "low_start": idxlows[0],
                                    "high_start":idxhighs[0],
                                    "idxlows": list(idxlows),
                                    "idxhighs": list(idxhighs),
                                    "lows":lows,
                                    "highs": highs,
                                    "slmin": slmin,
                                    "intercmin":intercmin,
                                    "rmin": rmin,
                                    "slmax": slmax,
                                    "intercmax": intercmax,
                                    "rmax":rmax
                                }
                            )
                    else:
                        ranges_list.append(
                            {
                                "low_start": idxlows[0],
                                "high_start":idxhighs[0],
                                "idxlows": list(idxlows),
                                "idxhighs": list(idxhighs),
                                "lows":lows,
                                "highs": highs,
                                "slmin": slmin,
                                "intercmin":intercmin,
                                "rmin": rmin,
                                "slmax": slmax,
                                "intercmax": intercmax,
                                "rmax":rmax
                            }
                        )
                        
            else:
                ranges_list.append(
                    {
                        "low_start": idxlows[0],
                        "high_start":idxhighs[0],
                        "idxlows": list(idxlows),
                        "idxhighs": list(idxhighs),
                        "lows":lows,
                        "highs": highs,
                        "slmin": slmin,
                        "intercmin":intercmin,
                        "rmin": rmin,
                        "slmax": slmax,
                        "intercmax": intercmax,
                        "rmax":rmax
                    }
                )

            return 1
    
    return 0

In [ ]:
from scipy.stats import linregress
ranges_list  = list()

def detect_flag(candle, backcandles, window, pivot_window, plot_flag=False):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    localdf = df[candle-backcandles-window:candle-window]  
    highs = list(localdf[localdf[pivot_window] == 2].high.tail(4).values)
    idxhighs = list(localdf[localdf[pivot_window] == 2].high.tail(4).index)

    if len(highs) == 4 and len(lows) == 4:        
        slmax, intercmax, rmax, _, _ = linregress(idxhighs, highs)
        if rmax > 0.9:
            ranges_last.update(
                {
                    "idxhighs": idxhighs,
                    "highs":highs_add,
                    "slmax": slmax_add,
                    "intercmax":intercmax_add,
                    "rmax": rmax_add,
                }
        else:
            slmax_add_c, intercmax_add_c, rmax_add_c, xmchoose, ymchoose = filter_inner_point(idxhighs_add, highs_add, type="high")
            if
        
        if (order_condition and 
            (rmax*rmax)>=0.5 

        ):

            if len(ranges_list) > 0:
                ranges_last = ranges_list[-1]
                
                if ranges_last["idxlows"][-1] == idxlows[-1] and ranges_last["idxhighs"][-1] == idxhighs[-1]:
                    pass
                else:

                        
                    elif ranges_last["idxhighs"][-1] == idxhighs[-2]:                        
                        idxhighs_add = ranges_last["idxhighs"] + [idxhighs[-1]]
                        highs_add = ranges_last["highs"] + [highs[-1]]
                        slmax_add, intercmax_add, rmax_add, _, _ = linregress(idxhighs_add, highs_add)
                        slmax_add_c, intercmax_add_c, rmax_add_c, xmchoose, ymchoose = filter_inner_point(idxhighs_add, highs_add, type="high")
                        print(xmchoose, rmax_add_c, ranges_last["rmax"])
                        
                        if (rmax_add*rmax_add)>=0.95:
                            ranges_last.update({
                                "idxhighs": idxhighs_add,
                                "highs":highs_add,
                                "slmax": slmax_add,
                                "intercmax":intercmax_add,
                                "rmax": rmax_add,
                        })
                        elif rmax_add_c > ranges_last["rmax"]:                            
                            ranges_last.update({
                                "idxhighs": xmchoose,
                                "highs":ymchoose,
                                "slmax": slmax_add_c,
                                "intercmax":intercmax_add_c,
                                "rmax": rmax_add_c,
                            }) 
                        else:
                            ranges_list.append(
                                {
                                    "low_start": idxlows[0],
                                    "high_start":idxhighs[0],
                                    "idxlows": list(idxlows),
                                    "idxhighs": list(idxhighs),
                                    "lows":lows,
                                    "highs": highs,
                                    "slmin": slmin,
                                    "intercmin":intercmin,
                                    "rmin": rmin,
                                    "slmax": slmax,
                                    "intercmax": intercmax,
                                    "rmax":rmax
                                }
                            )
                    else:
                        ranges_list.append(
                            {
                                "low_start": idxlows[0],
                                "high_start":idxhighs[0],
                                "idxlows": list(idxlows),
                                "idxhighs": list(idxhighs),
                                "lows":lows,
                                "highs": highs,
                                "slmin": slmin,
                                "intercmin":intercmin,
                                "rmin": rmin,
                                "slmax": slmax,
                                "intercmax": intercmax,
                                "rmax":rmax
                            }
                        )
                        
            else:
                ranges_list.append(
                    {
                        "low_start": idxlows[0],
                        "high_start":idxhighs[0],
                        "idxlows": list(idxlows),
                        "idxhighs": list(idxhighs),
                        "lows":lows,
                        "highs": highs,
                        "slmin": slmin,
                        "intercmin":intercmin,
                        "rmin": rmin,
                        "slmax": slmax,
                        "intercmax": intercmax,
                        "rmax":rmax
                    }
                )

            return 1
    
    return 0

In [330]:
# detect_flag(2189, 30, 3, plot_flag=True)

In [71]:
df_choose = df[20000:22000]

In [305]:
df_choose.low.tail(3).index[1]

3998

In [93]:
ranges_list

[{'low_start': 19775,
  'high_start': 19879,
  'idxlows': [19775, 19836, 19967],
  'idxhighs': [19879, 19880, 19919],
  'lows': [2.4906646498251486, 2.5103848766646193, 2.516300944716461],
  'highs': [2.558699432421323, 2.558699432421323, 2.566587523157111],
  'slmin': 0.0001194906551298948,
  'intercmin': 0.13277873995911849,
  'rmin': 0.8732942311100498,
  'slmax': 0.00019960255225088427,
  'intercmax': -1.409298241742838,
  'rmax': 0.9997597405162936},
 {'low_start': 19775,
  'high_start': 19919,
  'idxlows': [19775, 19836, 19967],
  'idxhighs': [19919, 19997, 20087],
  'lows': [2.4906646498251486, 2.5103848766646193, 2.516300944716461],
  'highs': [2.566587523157111, 2.558699432421323, 2.612930056229867],
  'slmin': 0.0001194906551298948,
  'intercmin': 0.13277873995911849,
  'rmin': 0.8732942311100498,
  'slmax': 0.00028732737573020816,
  'intercmax': -3.1723902353906897,
  'rmax': 0.7568132066994785},
 {'low_start': 19967,
  'high_start': 19997,
  'idxlows': [20018, 20154, 20155]

In [94]:
ranges_list = []
df_choose.index.map(lambda x: detect_flag(x, 320, 24, 'pivot_merge'))

[19754, 19919, 20012] -0.9082773075599568 -0.7579263939266307
[20012, 20087, 20159] 0.9871606257926565 0.9956749248218241
add_idx 20018
add_idx 20111
[20255, 20314, 20381] 0.5015215244623011 0.8738157726293816
add_idx 20236
add_idx 20644
add_idx 20687
add_idx 20687
[20639, 20736, 20887] -0.7572228726020105 0.7157469650876794
add_idx 20814
add_idx 20814
[21047, 21094, 21170] 0.9338187088138424 0.9836250343299465


Int64Index([1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
            ...
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           dtype='int64', length=2000)

In [67]:
ranges_list = []
df_choose.index.map(lambda x: detect_flag(x,240 , 24, 'pivot_merge'))

add_idx 192
add_idx 327
add_idx 521
add_idx 1027
add_idx 1086
add_idx 1248
add_idx 2275
add_idx 2275
add_idx 2275


/liubinxu/liubinxu/miniconda3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning:

invalid value encountered in double_scalars

/liubinxu/liubinxu/miniconda3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning:

invalid value encountered in sqrt

/liubinxu/liubinxu/miniconda3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning:

invalid value encountered in double_scalars



add_idx 3031
add_idx 3298
add_idx 3602


Int64Index([0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            ...
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           dtype='int64', length=4000)

In [393]:
[x["intercmax"] for x in ranges_list]

KeyError: 'intercmax'

In [87]:
linregress([911, 1027, 1086], [3.42271428, 3.36235925, 3.34483682])

LinregressResult(slope=-0.0004554804749590026, intercept=3.835761102092008, rvalue=-0.9926168156614321, pvalue=0.07740776807036429, stderr=5.565726842791014e-05, intercept_stderr=0.05624823523177259)

In [33]:
ranges_list

[{'low_start': 192,
  'high_start': 383,
  'idxlows': [192, 327, 407, 481],
  'idxhighs': [383, 432, 446, 489],
  'lows': [3.440236708619679,
   3.3896163608980436,
   3.375014337516799,
   3.3380225449509897],
  'highs': [3.440236708619665,
   3.424661217013005,
   3.3954571702505274,
   3.375987805742208],
  'slmin': -0.00033910674921420396,
  'intercmin': 3.505003287032474,
  'rmin': -0.9906781059325097,
  'slmax': -0.0006294811127493567,
  'intercmax': 3.6844837122341945,
  'rmax': -0.9536867008386456},
 {'low_start': 481,
  'high_start': 489,
  'idxlows': [481, 521, 577, 614],
  'idxhighs': [489, 528, 587, 644],
  'lows': [3.3380225449509897,
   3.3487306954305676,
   3.4071387889555176,
   3.4217408123367496],
  'highs': [3.375987805742208,
   3.3983775749267773,
   3.4519183273246457,
   3.4713876918329536],
  'slmin': 0.0006913052156965261,
  'intercmin': 2.9999001259128355,
  'rmin': 0.977039641524949,
  'slmax': 0.0006476652845689411,
  'intercmax': 3.060429960028901,
  'rmax

In [68]:
# df['flag'] = df.index.map(lambda x: detect_flag(x, 96, 6, ''))
ranges_list = list()
df['flag1'] = df.index.map(lambda x: detect_flag(x, 480, 24, 'pivot_merge'))

add_idx 327
add_idx 521
add_idx 1027
add_idx 1086
add_idx 1248
add_idx 2275
add_idx 2275
add_idx 2275
add_idx 2311


/liubinxu/liubinxu/miniconda3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:182: RuntimeWarning:

invalid value encountered in double_scalars

/liubinxu/liubinxu/miniconda3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:196: RuntimeWarning:

invalid value encountered in sqrt

/liubinxu/liubinxu/miniconda3/lib/python3.10/site-packages/scipy/stats/_stats_mstats_common.py:199: RuntimeWarning:

invalid value encountered in double_scalars



add_idx 3031
add_idx 3298
add_idx 3602
add_idx 4070
add_idx 4187
add_idx 4799
add_idx 5088
add_idx 5233
add_idx 5297
add_idx 6121
add_idx 6248
add_idx 6248
add_idx 6347
add_idx 6627
add_idx 6962
add_idx 6962
add_idx 6994
add_idx 7064
add_idx 7439
add_idx 7680
add_idx 8111
add_idx 8111
add_idx 8353
add_idx 8543
add_idx 8684
add_idx 10079
add_idx 10321
add_idx 10364
add_idx 10428
add_idx 10581
add_idx 11046
add_idx 11426
add_idx 11476
add_idx 11541
add_idx 11760
add_idx 11760
add_idx 11805
add_idx 11805
add_idx 11817
add_idx 12104
add_idx 12231
add_idx 12231
add_idx 12263
add_idx 12263
add_idx 12383
add_idx 12509
add_idx 12842
add_idx 12959
add_idx 12982
add_idx 13229
add_idx 14036
add_idx 14101
add_idx 14303
add_idx 14869
add_idx 14869
add_idx 14869
add_idx 14923
add_idx 15527
add_idx 15543
add_idx 15543
add_idx 15575
add_idx 15610
add_idx 16094
add_idx 16226
add_idx 16719
add_idx 16870
add_idx 16895
add_idx 16895
add_idx 16895
add_idx 16895
add_idx 17003
add_idx 17711
add_idx 18025
add

/tmp/ipykernel_3900674/1979567455.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [45]:
ranges_list

[{'low_start': 192,
  'high_start': 383,
  'idxlows': [192, 327, 407, 481],
  'idxhighs': [383, 432, 446, 489],
  'lows': [3.440236708619679,
   3.3896163608980436,
   3.375014337516799,
   3.3380225449509897],
  'highs': [3.440236708619665,
   3.424661217013005,
   3.3954571702505274,
   3.375987805742208],
  'slmin': -0.00033910674921420396,
  'intercmin': 3.505003287032474,
  'rmin': -0.9906781059325097,
  'slmax': -0.0006294811127493567,
  'intercmax': 3.6844837122341945,
  'rmax': -0.9536867008386456},
 {'low_start': 481,
  'high_start': 489,
  'idxlows': [481, 521, 577, 614],
  'idxhighs': [489, 528, 587, 644],
  'lows': [3.3380225449509897,
   3.3487306954305676,
   3.4071387889555176,
   3.4217408123367496],
  'highs': [3.375987805742208,
   3.3983775749267773,
   3.4519183273246457,
   3.4713876918329536],
  'slmin': 0.0006913052156965261,
  'intercmin': 2.9999001259128355,
  'rmin': 0.977039641524949,
  'slmax': 0.0006476652845689411,
  'intercmax': 3.060429960028901,
  'rmax

In [251]:
flags = df[df['flag']!=0]
flags[40:]

,open,high,low,close,volume,time,pivot,pointpos,flag,pivot24,pointpos24,pivot_merge,pointpos_merge
1139,3.362359,3.366253,3.361386,3.364306,17774300.0,2021-04-14 14:05:00,0,NaN,1,0,NaN,0,NaN
1140,3.364306,3.369174,3.364306,3.368200,55610000.0,2021-04-14 14:10:00,2,3.370174,1,0,NaN,0,NaN
1141,3.368200,3.368200,3.363333,3.366253,7915300.0,2021-04-14 14:15:00,0,NaN,1,0,NaN,0,NaN
1142,3.365280,3.366253,3.363333,3.365280,4179800.0,2021-04-14 14:20:00,0,NaN,1,0,NaN,0,NaN
1143,3.365280,3.365280,3.355545,3.356518,6139900.0,2021-04-14 14:25:00,0,NaN,1,0,NaN,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
23345,2.668000,2.668000,2.663000,2.665000,9895300.0,2023-03-13 11:05:00,0,NaN,1,0,NaN,0,NaN
23346,2.664000,2.668000,2.663000,2.668000,6919700.0,2023-03-13 11:10:00,0,NaN,1,0,NaN,0,NaN
23347,2.667000,2.668000,2.665000,2.667000,7653800.0,2023-03-13 11:15:00,0,NaN,1,0,NaN,0,NaN
23348,2.668000,2.668000,2.665000,2.667000,4502800.0,2023-03-13 11:20:00,0,NaN,1,0,NaN,0,NaN


In [371]:
ranges_list

[{'low_start': 157,
  'high_start': 257,
  'idxlows': [157, 192, 327, 407],
  'idxhighs': [257, 286, 338, 383, 432],
  'lows': [3.4489979226484206,
   3.440236708619679,
   3.3896163608980436,
   3.375014337516799],
  'highs': [3.5074060161733716,
   3.497671333919208,
   3.451918327324657,
   3.440236708619665,
   3.424661217013005],
  'slmin': -0.00031149792244389014,
  'intercmin': 3.4978043949224187,
  'rmin': -0.9923975554211752,
  'slmax': -0.0004988499667174194,
  'intermax': 3.6936078704048048,
  'rmax': -0.9741961176976569,
  'intercmax': 3.6335886293205295},
 {'low_start': 481,
  'high_start': 489,
  'idxlows': [481, 521, 577, 614],
  'idxhighs': [489, 528, 587, 644],
  'lows': [3.3380225449509897,
   3.3487306954305676,
   3.4071387889555176,
   3.4217408123367496],
  'highs': [3.375987805742208,
   3.3983775749267773,
   3.4519183273246457,
   3.4713876918329536],
  'slmin': 0.0006913052156965261,
  'intercmin': 2.9999001259128355,
  'rmin': 0.977039641524949,
  'slmax': 0.

In [98]:
flags =flags.loc[~flags.index.duplicated(keep='first')]

In [253]:
flags["last_index"] = [0] + list(flags.index[:-1])

/tmp/ipykernel_1333883/3038378998.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [170]:
df.loc[2000:2530,][df["pivot24"]!=0]

/tmp/ipykernel_1333883/3558862394.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,open,high,low,close,volume,time,pivot,pointpos,flag,pivot24,pointpos24
2000,3.306872,3.306872,3.296163,3.298110,15331900.0,2021-05-13 13:50:00,1,3.295163,0,1,3.295163
2015,3.326341,3.333155,3.320500,3.324394,22316300.0,2021-05-14 09:35:00,2,3.334155,0,2,NaN
2051,3.398378,3.403245,3.393510,3.395457,13258300.0,2021-05-14 14:05:00,2,3.404245,0,2,NaN
2069,3.448998,3.457759,3.445104,3.450945,17709900.0,2021-05-17 10:05:00,2,3.458759,0,2,3.458759
2080,3.438290,3.446078,3.438290,3.445104,6387900.0,2021-05-17 11:00:00,1,NaN,0,1,NaN
2083,3.443157,3.443157,3.438290,3.441210,7844200.0,2021-05-17 11:15:00,1,3.437290,0,1,NaN
2087,3.440237,3.447051,3.439263,3.445104,9816900.0,2021-05-17 13:05:00,3,3.448051,0,1,NaN
2090,3.441210,3.441210,3.429529,3.434396,19336100.0,2021-05-17 13:20:00,1,3.428529,0,1,NaN
2096,3.435369,3.439263,3.432449,3.438290,9269800.0,2021-05-17 13:50:00,1,NaN,0,1,NaN
2099,3.436343,3.436343,3.432449,3.434396,5483200.0,2021-05-17 14:05:00,1,3.431449,0,1,NaN


In [258]:
flags.to_csv("flags.tsv", sep="\t")

In [254]:
flags[flags.index - flags["last_index"]  != 1]

,open,high,low,close,volume,time,pivot,pointpos,flag,pivot24,pointpos24,pivot_merge,pointpos_merge,last_index
514,3.354572,3.358465,3.353598,3.356518,4495100.0,2021-03-25 14:00:00,0,NaN,1,0,NaN,0,NaN,0
1131,3.348731,3.351651,3.347757,3.351651,6684200.0,2021-04-14 13:25:00,0,NaN,1,0,NaN,0,NaN,545
2521,3.578469,3.583337,3.574575,3.580416,17266000.0,2021-05-28 13:15:00,0,NaN,1,0,NaN,0,NaN,1196
3471,3.429529,3.431475,3.428555,3.429529,2748200.0,2021-06-28 10:55:00,0,NaN,1,0,NaN,0,NaN,2544
4776,3.173506,3.173506,3.166692,3.166692,7994600.0,2021-08-04 13:10:00,0,NaN,1,0,NaN,0,NaN,3526
4876,3.139435,3.141382,3.135541,3.135541,2450000.0,2021-08-06 13:30:00,0,NaN,1,0,NaN,0,NaN,4823
5968,3.178374,3.179347,3.176427,3.177400,5838900.0,2021-09-08 11:00:00,0,NaN,1,0,NaN,0,NaN,4919
7786,3.097576,3.101470,3.096602,3.098549,17128800.0,2021-11-10 10:30:00,0,NaN,1,0,NaN,0,NaN,6048
10746,3.085229,3.085229,3.081285,3.083257,8888000.0,2022-02-11 14:40:00,0,NaN,1,0,NaN,0,NaN,7819
10929,3.081285,3.082271,3.079313,3.080299,4443100.0,2022-02-17 13:55:00,0,NaN,1,0,NaN,0,NaN,10750


In [52]:
a = 20
flags[a:a+20]

,open,high,low,close,volume,time,pivot,pointpos,flag
725,3.446078,3.451918,3.446078,3.450945,3470600.0,2021-04-01 10:05:00,0,NaN,1
726,3.449971,3.453865,3.448024,3.452892,12855700.0,2021-04-01 10:10:00,0,NaN,1
727,3.452892,3.462626,3.452892,3.461653,7610700.0,2021-04-01 10:15:00,0,NaN,1
728,3.460680,3.468467,3.460680,3.462626,11731600.0,2021-04-01 10:20:00,2,3.469467,1
729,3.462626,3.465547,3.461653,3.461653,6616600.0,2021-04-01 10:25:00,0,NaN,1
730,3.461653,3.464573,3.459706,3.459706,2507400.0,2021-04-01 10:30:00,0,NaN,1
1223,3.329261,3.331208,3.326341,3.330235,15026300.0,2021-04-16 13:05:00,0,NaN,1
1224,3.329261,3.334129,3.328288,3.329261,22048700.0,2021-04-16 13:10:00,0,NaN,1
1225,3.329261,3.330235,3.327314,3.329261,11652300.0,2021-04-16 13:15:00,0,NaN,1
1226,3.329261,3.332182,3.329261,3.330235,12858800.0,2021-04-16 13:20:00,0,NaN,1
